# 1. CNN



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 데이터 전처리

In [ ]:
X_train_processing = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=25, # 0~25도 사이에서 임의의 각도로 원본이미지를 회전
                                   width_shift_range=0.05, # 0.05범위 내에서 임의의 값만큼 임의의 방향으로 좌우 이동
                                   height_shift_range=0.05, # 0.05범위 내에서 임의의 값만큼 임의의 방향으로 상하 이동
                                   zoom_range=0.2, # (1-0.2)~(1+0.2) => 0.8~1.2 사이에서 임의의 수치만큼 확대/축소
                                   horizontal_flip=True, # 좌우로 뒤집기                                   
                                   vertical_flip=True,
                                   fill_mode='nearest'
                                  ) 
X_validation_processing = ImageDataGenerator(rescale = 1./255)
X_test_processing = ImageDataGenerator(rescale = 1./255)

In [ ]:
#w/o poisoning
#train_dir = './xray_undersampling/train/'
#test_dir = './xray_undersampling/test/'

In [ ]:
#poisoning dataset
train_dir = './50%/train/'
test_dir = './50%/test/'

In [ ]:
X_train = X_train_processing.flow_from_directory(train_dir, 
                                                    batch_size=16, # 한번에 변환된 이미지 16개씩 만들어라 라는 것
                                                    color_mode='grayscale', # grayscale: 흑백 이미지 처리
                                                    class_mode='binary', 
                                                    target_size=(150,150)) # target_size에 맞춰서 이미지의 크기가 조절된다

X_test = X_test_processing.flow_from_directory(test_dir,
                                                  batch_size=4,
                                                  color_mode='grayscale',
                                                  class_mode='binary',
                                                  target_size=(150,150))

Found 2682 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
X_train.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

## 모델 구성

In [ ]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential

from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[150,150, 1]), #커널 7x7, 필터 64개
    keras.layers.MaxPooling2D(pool_size=2), #크기 1/2로 줄어듦
    DefaultConv2D(filters=128), #나머지는 default로
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(), #중간에 flatten
    keras.layers.Dense(units=128, activation='relu'), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1, activation='sigmoid'),
])

## 기본 이진분류 & 라벨 스무딩

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0), #label_smoothing 값 바꿔주기
    optimizer=RMSprop(lr=1e-4),
    metrics=['acc']
)


/home/agent/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(
    X_train,
    steps_per_epoch=10,
    epochs=30,
    validation_steps=50
)

<ipython-input-14-bbda1bd4caff>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
10/10 [==============================] - 18s 1s/step - loss: 0.7080 - acc: 0.4812
Epoch 2/30
10/10 [==============================] - 13s 1s/step - loss: 0.6942 - acc: 0.5562
Epoch 3/30
10/10 [==============================] - 13s 1s/step - loss: 0.6970 - acc: 0.4625
Epoch 4/30
10/10 [==============================] - 13s 1s/step - loss: 0.6967 - acc: 0.5000
Epoch 5/30
10/10 [==============================] - 12s 1s/step - loss: 0.6944 - acc: 0.4437
Epoch 6/30
10/10 [==============================] - 12s 1s/step - loss: 0.6943 - acc: 0.4250
Epoch 7/30
10/10 [==============================] - 12s 1s/step - loss: 0.6933 - acc: 0.4875
Epoch 8/30
10/10 [==============================] - 12s 1s/step - loss: 0.6920 - acc: 0.5250
Epoch 9/30
10/10 [==============================] - 12s 1s/step - loss: 0.6955 - acc: 0.4563
Epoch 10/30
10/10 [==============================] - 12s 1s/step - loss: 0.6938 - acc: 0.5375
Epoch 11/30
10/10 [==============================] - 12s 1s/step - lo

In [ ]:
# 모델 성능 평가
model.evaluate(X_test)

156/156 [==============================] - 11s 71ms/step - loss: 0.6925 - acc: 0.6250


[0.6924883723258972, 0.625]

## focal loss

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryFocalCrossentropy(),
    optimizer=RMSprop(lr=1e-4),
    metrics=['acc']
)


In [ ]:
history = model.fit_generator(
    X_train,
    steps_per_epoch=10, 
    epochs=30,
    validation_steps=50
)

Epoch 1/30


<ipython-input-17-50da17d6b332>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


10/10 [==============================] - 13s 1s/step - loss: 0.1740 - acc: 0.4875
Epoch 2/30
10/10 [==============================] - 11s 1s/step - loss: 0.1734 - acc: 0.4805
Epoch 3/30
10/10 [==============================] - 12s 1s/step - loss: 0.1735 - acc: 0.4750
Epoch 4/30
10/10 [==============================] - 12s 1s/step - loss: 0.1734 - acc: 0.4500
Epoch 5/30
10/10 [==============================] - 12s 1s/step - loss: 0.1727 - acc: 0.5063
Epoch 6/30
10/10 [==============================] - 12s 1s/step - loss: 0.1739 - acc: 0.4938
Epoch 7/30
10/10 [==============================] - 12s 1s/step - loss: 0.1739 - acc: 0.4688
Epoch 8/30
10/10 [==============================] - 11s 1s/step - loss: 0.1730 - acc: 0.5437
Epoch 9/30
10/10 [==============================] - 12s 1s/step - loss: 0.1727 - acc: 0.5813
Epoch 10/30
10/10 [==============================] - 11s 1s/step - loss: 0.1724 - acc: 0.5750
Epoch 11/30
10/10 [==============================] - 12s 1s/step - loss: 0.1737 

In [ ]:
# 모델 성능 평가
model.evaluate(X_test)

156/156 [==============================] - 12s 78ms/step - loss: 0.1702 - acc: 0.6250


[0.17023253440856934, 0.625]

# 2. DNN

## 데이터 전처리

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
from PIL import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms
import os
import cv2

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
X_train_processing = ImageDataGenerator(rescale = 1./255,
                                        width_shift_range=0.05,
                                        height_shift_range=0.05,
                                        zoom_range=0.2,
                                        vertical_flip=True,
                                        fill_mode='nearest'
                                      )
X_validation_processing = ImageDataGenerator(rescale = 1./255)
X_test_processing = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_dir = '20%/train'
X_train = X_train_processing.flow_from_directory(train_dir,
                                                batch_size=16,
                                                color_mode='grayscale',
                                                class_mode='binary',
                                                target_size=(150,150))

test_dir = "20%/test"
X_test = X_test_processing.flow_from_directory(test_dir,
                                                batch_size=4,
                                                color_mode='grayscale',
                                                class_mode='binary',
                                                target_size=(150,150))

Found 2682 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## 모델 구성

In [ ]:
#모델 생성
input_dim = 1

def create_model():
  model = tf.keras.Sequential()
  #레이어 추가
  model.add(tf.keras.layers.Flatten(input_shape=(150, 150, input_dim)))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dense(64, activation='relu'))

  #출력층
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  return model

## 기본 이진 분류 & 라벨 스무딩

In [ ]:
model = create_model()

model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy, #loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0) 사용해서 Lable Smoothing 값 바꿔주기 
              metrics=['accuracy'])

In [ ]:
model.fit(
    X_train,
    epochs=30
)

Epoch 1/30
168/168 [==============================] - 461s 3s/step - loss: 0.7533 - accuracy: 0.5955
Epoch 2/30
168/168 [==============================] - 51s 303ms/step - loss: 0.7231 - accuracy: 0.5969
Epoch 3/30
168/168 [==============================] - 50s 299ms/step - loss: 0.6398 - accuracy: 0.6380
Epoch 4/30
168/168 [==============================] - 51s 302ms/step - loss: 0.6404 - accuracy: 0.6480
Epoch 5/30
168/168 [==============================] - 51s 304ms/step - loss: 0.6480 - accuracy: 0.6167
Epoch 6/30
168/168 [==============================] - 50s 299ms/step - loss: 0.6469 - accuracy: 0.6480
Epoch 7/30
168/168 [==============================] - 50s 296ms/step - loss: 0.6293 - accuracy: 0.6514
Epoch 8/30
168/168 [==============================] - 50s 297ms/step - loss: 0.6203 - accuracy: 0.6723
Epoch 9/30
168/168 [==============================] - 51s 307ms/step - loss: 0.6249 - accuracy: 0.6611
Epoch 10/30
168/168 [==============================] - 50s 297ms/step - los

In [ ]:
print("-- Evaluate --")
test_loss, test_acc = model.evaluate_generator(X_test)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

-- Evaluate --


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  



Test accuracy: 0.8381410241127014

Test loss: 0.5013676285743713


## focal loss

In [ ]:
model = create_model()

model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_focal_crossentropy, 
              metrics=['accuracy'])

In [ ]:
model.fit(
    X_train,
    epochs=30
)

Epoch 1/30
168/168 [==============================] - 51s 298ms/step - loss: 0.4734 - accuracy: 0.5645
Epoch 2/30
168/168 [==============================] - 53s 316ms/step - loss: 0.2308 - accuracy: 0.5705
Epoch 3/30
168/168 [==============================] - 51s 302ms/step - loss: 0.1842 - accuracy: 0.6167
Epoch 4/30
168/168 [==============================] - 50s 300ms/step - loss: 0.1685 - accuracy: 0.6066
Epoch 5/30
168/168 [==============================] - 51s 305ms/step - loss: 0.1601 - accuracy: 0.6499
Epoch 6/30
168/168 [==============================] - 50s 300ms/step - loss: 0.1638 - accuracy: 0.6283
Epoch 7/30
168/168 [==============================] - 51s 301ms/step - loss: 0.1597 - accuracy: 0.6518
Epoch 8/30
168/168 [==============================] - 51s 304ms/step - loss: 0.1681 - accuracy: 0.6350
Epoch 9/30
168/168 [==============================] - 52s 307ms/step - loss: 0.1606 - accuracy: 0.6424
Epoch 10/30
168/168 [==============================] - 50s 300ms/step - l

In [ ]:
print("-- Evaluate --")
test_loss, test_acc = model.evaluate_generator(X_test)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

-- Evaluate --


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  



Test accuracy: 0.7564102411270142

Test loss: 0.15852375328540802
